<a href="https://colab.research.google.com/github/czek0/Data-Science-Portfolio/blob/main/Fuel_Efficiency.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Driving Visual Anaylses with Automobile Data
This dataset, available at http://www.fueleconomy. gov/feg/epadata/vehicles.csv.zip, contains fuel efficiency performance metrics over time for all makes and models of automobiles in the United States of America. This dataset also contains numerous other features and attributes of the automobile models other than fuel economy, providing an opportunity to summarize and group the data so that we can identify interesting trends and relationships.

In [10]:
import numpy as np


In [1]:
import pandas as pd
import requests
import io
import zipfile

url = 'http://fueleconomy.gov/feg/epadata/vehicles.csv.zip'
r = requests.get(url)
zipfile_data = io.BytesIO(r.content)

with zipfile.ZipFile(zipfile_data, 'r') as zip_ref:
    csv_file = zip_ref.open(zip_ref.namelist()[0])

df = pd.read_csv(csv_file)

<ipython-input-1-71ed092dffce>:13: DtypeWarning: Columns (72,74,75,77) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv_file)


In [17]:
df.head()
df.shape

(47523, 84)

This tells us that columns 22, 23, 70, 71, 72, and 73 contain mixed data types. Let's find the corresponding names using the following commands:

In [14]:
column_names = df.columns.values
column_names[[22, 23, 70, 71, 72, 73]]


array(['cylinders', 'displ', 'atvType', 'fuelType2', 'rangeA', 'evMotor'],
      dtype=object)

##Preliminary analysis

In [23]:
print("length: ",len(df))
print("collumns: ", len(df.columns))
print(df.columns)

length:  47523
collumns:  84
Index(['barrels08', 'barrelsA08', 'charge120', 'charge240', 'city08',
       'city08U', 'cityA08', 'cityA08U', 'cityCD', 'cityE', 'cityUF', 'co2',
       'co2A', 'co2TailpipeAGpm', 'co2TailpipeGpm', 'comb08', 'comb08U',
       'combA08', 'combA08U', 'combE', 'combinedCD', 'combinedUF', 'cylinders',
       'displ', 'drive', 'engId', 'eng_dscr', 'feScore', 'fuelCost08',
       'fuelCostA08', 'fuelType', 'fuelType1', 'ghgScore', 'ghgScoreA',
       'highway08', 'highway08U', 'highwayA08', 'highwayA08U', 'highwayCD',
       'highwayE', 'highwayUF', 'hlv', 'hpv', 'id', 'lv2', 'lv4', 'make',
       'model', 'mpgData', 'phevBlended', 'pv2', 'pv4', 'range', 'rangeCity',
       'rangeCityA', 'rangeHwy', 'rangeHwyA', 'trany', 'UCity', 'UCityA',
       'UHighway', 'UHighwayA', 'VClass', 'year', 'youSaveSpend', 'baseModel',
       'guzzler', 'trans_dscr', 'tCharger', 'sCharger', 'atvType', 'fuelType2',
       'rangeA', 'evMotor', 'mfrCode', 'c240Dscr', 'charge240b', 'c

Let's find out how many unique years of data are included in this dataset and what the first and last years are using the following command:

In [24]:
print("Unique years data: ", pd.unique(df.year))
print("Min year: ", min(df.year))
print("Max year: ", max(df.year))


Unique years data:  [1985 1993 1994 1995 1996 1997 1998 1999 2000 2001 1986 2002 2003 2004
 2005 2006 2007 2008 2009 2010 1984 1987 1988 1989 1990 1991 1992 2011
 2012 2013 2014 2015 2016 2017 2018 2019 2020 2021 2022 2023 2024 2025]
Min year:  1984
Max year:  2025


Let's find out what types of fuel are used as the automobiles' primary fuel types.

In [25]:
print("Types of fuel", pd.value_counts(df.fuelType1))

Types of fuel fuelType1
Regular Gasoline     30426
Premium Gasoline     14801
Diesel                1274
Electricity            766
Midgrade Gasoline      164
Natural Gas             60
Hydrogen                32
Name: count, dtype: int64


Now if we want to explore what types of transmissions these automobiles have, we immediately try the following command:

In [26]:
print("Types of transmissions:", pd.value_counts(df.trany))

Types of transmissions: trany
Automatic 4-spd                     11048
Manual 5-spd                         8391
Automatic (S6)                       3337
Automatic (S8)                       3157
Automatic 3-spd                      3151
Manual 6-spd                         3079
Automatic 5-spd                      2203
Automatic 6-spd                      1752
Manual 4-spd                         1483
Automatic (variable gear ratios)     1104
Automatic 8-spd                       978
Automatic (S5)                        848
Automatic (AM-S7)                     763
Automatic 9-spd                       745
Automatic (A1)                        730
Automatic 7-spd                       720
Automatic (S10)                       588
Automatic (S7)                        386
Automatic 10-spd                      380
Automatic (AV-S6)                     324
Automatic (AM7)                       318
Automatic (AM-S8)                     285
Automatic (S9)                        250
Auto

What we really want to know is the number of cars with automatic and manual transmissions. We notice that the trany variable always starts with the letter A when it represents an automatic transmission and M for manual transmission. Thus, we create a new variable, trany2, that contains the first character of the trany variable, which is a string:


In [27]:
df["trany2"] = df.trany.str[0]
print("New Types of Transmissions: ", pd.value_counts(df.trany2))

New Types of Transmissions:  trany2
A    34289
M    13223
Name: count, dtype: int64
